<a href="https://colab.research.google.com/github/marrvinn/marrvinn/blob/main/Schneidzeit/01_cut_time_preprocessing_marv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TODO

  * [done] sub_bin_maker zuweisen der schnitte zu den flyern der subbins
  * [done] endlossschleife, da von longest_bin ein bin mit 0 elementen abgeschnitten wird --> Lösung mit actual_cut nicht ausreichend, da schnitte für subbins als durchgeführt angesehen werden, obwohl die subbins noch nicht geschnitten sind --> es verbleiben subbins mit der länge > 1, die nicht geschnitten sind 

* [done] Rand wegschneiden
  * [done] zuerst die einfachste Option --> wenn 1 Element --> Rand weg
  * besser in Zukunft --> Rand wegschneiden bevor auseinander geschnitten, um Schnitte zu sparen evtl. mit Heuristik anhand der Fläche tatsächlich vs bedruckt bestimmen

* (nicht) optimale Schnittanzahl ausgeben (d.h. jeden Sub-Bin einzeln schneiden), wenn Sub-Bins nicht gleichzeitig geschnitten (bzw. sie werden nebeneinander gelegt)

* Schnittreihenfolge optimieren, kurzfristige Verschlechterung, um später im SChnittprozess weniger Schnitte zu benörigen, siehe Beispiel ppt

* Verbesserung in calc_nor_margin Werte in abin/df schreiben - besser durch .loc o.ä, um wirklich nur die Koordinaten der entsprechenden JobID zu ersetzen

* optional: margin_cuts direkt zu Beginn in abin schreiben

# Import and Install

In [33]:
import pandas as pd
import numpy as np
import random
import re
import glob
import matplotlib.patches as patches
import warnings
import sys
from matplotlib import pyplot as plt
from os import listdir
from os.path import isfile, join
from pathlib import Path
from pandas.core.reshape.merge import merge
M = 50000    # big M ist eine sehr große Zahl

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
data_path = '/content/drive/MyDrive/PJS/01_Input/Daten_v1_clean/00_Eingelesen/flyerdata_170_ma_a18.csv'

In [36]:
df = pd.DataFrame(pd.read_csv(data_path, sep=',', header=0))
# df = df.drop(['Unnamed: 0'], axis=1)

In [37]:
df.dtypes

DocName         object
x_left         float64
x_right        float64
y_bot          float64
y_top          float64
x_sheetsize    float64
y_sheetsize    float64
BSig            object
MarBot         float64
MarLeft        float64
MarRight       float64
MarTop         float64
Orient          object
JobID            int64
ProjectID        int64
day            float64
month          float64
year           float64
ProductPart     object
Priority         int64
amount           int64
cut_time       float64
cuts             int64
layer            int64
cut_layer        int64
dtype: object

# Basic Preprocessing

## get_abin

In [38]:
def get_abin(df, job_ID):
  abin = df.loc[df['JobID'] == job_ID]
  return abin

## get_format

In [39]:
def get_format(data_path, df):
  split1 = data_path.split('/')
  path_name = split1[-1]
  split2 = path_name.split('.')
  path_name = split2[0]
  split3 = path_name.split('_')
  format = split3[-1]
  grammatur = split3[1]

  # Hinzufügen des Formats und der Grammatur in den df
  df['format'] = format
  df['grammatur'] = grammatur
  df['grammatur'] = pd.to_numeric(df['grammatur'])
  return df

## calc_layer

In [40]:
def calc_layer(df):
  df['new_layer'] = (df['grammatur'] / 1000)
  df['new_layer'] = df['new_layer']*df['amount']
  df['new_layer'] = (df['new_layer']/10)
  df['new_layer'] = (df['new_layer']/12)
  df['new_layer'] = round(df['new_layer'] + .5)

  return df

## round_df

In [41]:
def round_df(df):
  df[['x_left', 'x_right', 'y_bot', 'y_top', 'x_sheetsize',	'y_sheetsize', 'MarBot', 'MarLeft',	'MarRight',	'MarTop']] = df[['x_left', 'x_right', 'y_bot', 'y_top', 'x_sheetsize',	'y_sheetsize', 'MarBot', 'MarLeft',	'MarRight',	'MarTop']].round(decimals = 0)
  return df

## calc_measures

In [42]:
def calc_measures(df):
  df['width'] = abs(df['x_left'] - df['x_right'])
  df['height'] = abs(df['y_bot'] - df['y_top'])
  return df

## Funktionsaufruf

In [43]:
def prep_df(df):
  df = get_format(data_path, df)
  df = round_df(df)
  df = calc_measures(df)
  df = calc_layer(df)

In [44]:
prep_df(df)

# Margins

## clear_overlap_deprecated

In [45]:
# # Überlappungen unter Einbeziehung der Margin berechnen
# # - Warum? Zur Feststellung ob nur ein Schnitt (Überlappung durch Margin) oder zwei Schnitt (tatsächliche Überlappung)
# # nach Abstimmung mit Flyeralarm:
# # kann vernachlässigt werden -> saubere Berechnung Flyer, Annahme bei Überlappung cut in der Mitte

# def clear_x_overlap(abin, df):

#     # integer status variables for value setting
#     abin['status_x_left'] = 0
#     abin['status_x_right'] = 0
#     abin['status_y_bot'] = 0
#     abin['status_y_top'] = 0

#     # vergleiche die Koordinaten nebeneinander liegender Flyer 
#     for index, row in abin.iterrows():

#         # zwischenspeichern aktuelle Flyer Daten
#         actual_flyer = row['DocName']
#         x_left = row['x_left']
#         x_right = row['x_right']
#         y_bot = row['y_bot']
#         y_top = row['y_top']
        
#         print('--- neuer Ausgangsflyer:', actual_flyer)

#         # listen um mehrere overlaps mit dem aktuellen Flyer zu speichern
#         list_avg_x_overlap_right = []
#         list_avg_x_overlap_left = []      

#         # der aktuelle Flyer wird mit allen anderen Flyern verglichen (inkl. sich selbst, was egal ist, da keine Überlappung)
#         for index, row in abin.iterrows():
#             comp_flyer = row['DocName']
#             x_comp_left = row['x_left']
#             x_comp_right = row['x_right']
#             y_comp_bot = row['y_bot']
#             y_comp_top = row['y_top']

#             # wenn die Flyer zumindestens teilweise auf der gleichen Höhe liegen, kann eine Überlappung vorliegen
#             if (((y_comp_top < y_top) & (y_comp_top > y_bot)) |
#                 ((y_comp_bot < y_top) & (y_comp_bot > y_bot)) |
#                 ((y_comp_bot == y_bot) & (y_comp_top == y_top))):

#                 print('--- auf Höhe')
#                 print('comp_flyer', comp_flyer)
#                 # print('x_comp_left', x_comp_left)
#                 # print('x_comp_right', x_comp_right)
#                 # print('x_left', x_left)
#                 # print('x_right', x_right)

#                 # berechne etwaige Überlappung rechtsseitig des Ausgangsflyers 
#                 x_overlap_right = x_right - x_comp_left
#                 # - analog linksseitig 
#                 x_overlap_left = x_comp_right - x_left

#                 # Berechne die Mitte der Überlappung rechtsseitig des Ausgangsflyers, insofern diese wirklich eine ist
#                 if ((x_right > x_comp_left) &
#                     (x_right < x_comp_right) & 
#                     (x_overlap_right > 0)):

#                     # easy math
#                     avg_x_overlap_right = round((x_right + x_comp_left)/2, ndigits=0)

#                     # set status 1 for list_avg_x_overlap_right, if status is 0
#                     if abin.loc[abin['DocName'] == comp_flyer, 'status_x_left'].item() == 0:
                                             
#                         # save in list
#                         list_avg_x_overlap_right.append(avg_x_overlap_right)

#                         print('list_avg_x_overlap_right', list_avg_x_overlap_right)
#                         print('avg_x_overlap_right', avg_x_overlap_right)

#                         # update status
#                         abin.loc[abin['DocName'] == comp_flyer, 'status_x_left'] = 1 
                    
#                     # instead use existing overlap
#                     elif abin.loc[abin['DocName'] == comp_flyer, 'status_x_left'].item() == 2:
#                         # save value of status 2 flyer instead
#                         instead_x_overlap_right = abin.loc[abin['DocName'] == comp_flyer, 'x_left']
#                         list_avg_x_overlap_right.append(instead_x_overlap_right)

#                 # - analog linksseitig
#                 if ((x_left < x_comp_right) &
#                     (x_left > x_comp_left) & 
#                     (x_overlap_left > 0)):

#                     # easy math
#                     avg_x_overlap_left = round((x_overlap_left + x_left)/2, ndigits=0)

#                     # set status 1 for list_avg_x_overlap_left
#                     if abin.loc[abin['DocName'] == comp_flyer, 'status_x_right'].item() == 0:
                        
#                         # save in list
#                         list_avg_x_overlap_left.append(avg_x_overlap_left)

#                         # update status
#                         abin.loc[abin['DocName'] == comp_flyer, 'status_x_right'] = 1
                    
#                     # instead use existing overlap
#                     elif abin.loc[abin['DocName'] == comp_flyer, 'status_x_right'].item() == 2:
#                         # save value of status 2 flyer instead
#                         instead_x_overlap_right = abin.loc[abin['DocName'] == comp_flyer, 'x_right']
#                         list_avg_x_overlap_right.append(instead_x_overlap_right)



# # Austausch der Werte im abin
        
#         # Durchschnittswerte der Listen setzen, nur wenn len(list) != 0 --> Überlappungen vorhanden 
#         if list_avg_x_overlap_right:
#             avg_avg_x_overlap_right = (sum(list_avg_x_overlap_right) / len(list_avg_x_overlap_right))

#             # - für den Ausgangsflyer, if len(list) != 0 --> Überlappungen gefunden, Werte müssen ersetzt werden
#             abin.loc[abin['DocName'] == actual_flyer, 'x_right'] = avg_avg_x_overlap_right
#             abin.loc[abin['DocName'] == actual_flyer, 'status_x_right'] = 2

#             # - für die Compare Flyer
#             # for index, row in abin.iterrows():
#             # check which status variables for left side of compare flyer are 1, if true set value avg_avg_x_overlap_right
#             abin.loc[abin['status_x_left'] == 1, 'x_left'] = avg_avg_x_overlap_right
#             abin.loc[abin['status_x_left'] == 1, 'status_x_left'] = 2

#         # Durchschnittswerte der Listen setzen, nur wenn len(list) != 0 --> Überlappungen vorhanden 
#         if list_avg_x_overlap_left:
#             avg_avg_x_overlap_left = sum(list_avg_x_overlap_left) / len(list_avg_x_overlap_left)
 
#             # - für den Ausgangsflyer, if len(list) != 0 --> Überlappungen gefunden, Werte müssen ersetzt werden
#             abin.loc[abin['DocName'] == actual_flyer, 'x_left'] = avg_avg_x_overlap_left
#             abin.loc[abin['DocName'] == actual_flyer, 'status_x_left'] = 2

#             # - für die Compare Flyer
#             # for index, row in abin.iterrows():
#             # check which status variables for left side of compare flyer are 1, if true set value avg_avg_x_overlap_left
#             abin.loc[abin['status_x_right'] == 1, 'x_right'] = avg_avg_x_overlap_left
#             abin.loc[abin['status_x_right'] == 1, 'status_x_right'] = 2



#     # Update der Koordinaten im df, push abin in df
    
#     # ...
    
#     # Drop status cols
#     # abin.drop(['status_x_left', 'status_x_right', 'status_y_bot', 'status_y_top'], axis = 1, inplace=True)

## check_overlap

In [46]:
def check_overlap(abin):

    checker = None
    list_checker = [] 

    # vergleiche die Koordinaten nebeneinander liegender Flyer 
    for index, row in abin.iterrows():

        # zwischenspeichern aktuelle Flyer Daten
        actual_flyer = row['DocName']
        x_left = row['x_left']
        x_right = row['x_right']
        y_bot = row['y_bot']
        y_top = row['y_top']

        # der aktuelle Flyer wird mit allen anderen Flyern verglichen (inkl. sich selbst, was egal ist, da keine Überlappung)
        for index, row in abin.iterrows():
            comp_flyer = row['DocName']
            x_comp_left = row['x_left']
            x_comp_right = row['x_right']
            y_comp_bot = row['y_bot']
            y_comp_top = row['y_top']

            # wenn die Flyer zumindestens teilweise auf der gleichen Höhe liegen, kann eine Überlappung vorliegen
            if (((y_comp_top < y_top) & (y_comp_top > y_bot)) |
                ((y_comp_bot < y_top) & (y_comp_bot > y_bot)) |
                ((y_comp_bot == y_bot) & (y_comp_top == y_top))):

                # berechne etwaige Überlappung rechtsseitig des Ausgangsflyers 
                x_overlap_right = x_right - x_comp_left
                # - analog linksseitig 
                x_overlap_left = x_comp_right - x_left

                # Überlappung rechtsseitig des Ausgangsflyers, insofern diese wirklich eine ist
                if ((x_right > x_comp_left) &
                    (x_right < x_comp_right) & 
                    (x_overlap_right > 0)):

                    # alarm
                    checker = True
                    list_checker.append(checker)

                # - analog linksseitig
                if ((x_left < x_comp_right) &
                    (x_left > x_comp_left) & 
                    (x_overlap_left > 0)):

                    # alarm
                    checker = True
                    list_checker.append(checker)

    if True in list_checker:
        check = True
    else:
        check = False

    return check

## clear_margin

In [47]:
def clear_margin(abin):
    # verrechnen der Margins unter der Annahme, dass die Daten korrekt sind
    abin['x_left'] = abin.apply(lambda row: row.x_left + (row.MarLeft/2), axis=1)
    abin['x_right'] = abin.apply(lambda row: row.x_right - (row.MarRight/2), axis=1)
    abin['y_bot'] = abin.apply(lambda row: row.y_bot + (row.MarBot/2), axis=1)
    abin['y_top'] = abin.apply(lambda row: row.y_top - (row.MarTop/2), axis=1)

## margin_cleaner

In [48]:
def margin_cleaner(abin, df):

    for i in range(6):
        checker = check_overlap(abin)
        if checker == True:
            print('checker:', checker)
            clear_margin(abin)

    print('end of for')

    del checker
    # check again after clear_margin for i times
    checker = check_overlap(abin)

    if checker == False:
        print('Awesome, no margin left! Lets update the df :)')

        # if False, update all flyer coordinates from abin into df
        for index, row in abin.iterrows(): 
            df.at[df['DocName'] == row['DocName'], 'x_left'] = row['x_left']
            df.at[df['DocName'] == row['DocName'], 'x_right'] = row['x_right']
            df.at[df['DocName'] == row['DocName'], 'y_bot'] = row['y_bot']
            df.at[df['DocName'] == row['DocName'], 'y_top'] = row['y_top']

    elif checker == True:
        print('Oink this data is incredible! Lets exclude the bad data and move on :)')
        
        # if still True, failure in data, break masterfunction and exclude data from df
        # ...

    else:
        sys.exit("An Error occured! There is no reasonable value for check_overlap during margin cleaning.")

In [49]:
abin = get_abin(df, 6667060114)
abin

# 6667060114 mit overlap
# 6593636202 ohne overlap

,DocName,x_left,x_right,y_bot,y_top,x_sheetsize,y_sheetsize,BSig,MarBot,MarLeft,...,amount,cut_time,cuts,layer,cut_layer,format,grammatur,width,height,new_layer
303,6667060114_0,1854.0,2469.0,40.0,896.0,2494.0,1786.0,X2,3.0,3.0,...,1000,0.319,26,2,13,a18,170,615.0,856.0,2.0
304,6667060114_1,1854.0,2469.0,890.0,1746.0,2494.0,1786.0,X3,3.0,3.0,...,1000,0.319,26,2,13,a18,170,615.0,856.0,2.0
305,6667060114_2,26.0,641.0,43.0,1746.0,2494.0,1786.0,R1,3.0,3.0,...,1000,0.319,26,2,13,a18,170,615.0,1703.0,2.0
306,6667060114_3,635.0,1250.0,34.0,896.0,2494.0,1786.0,Z1,3.0,3.0,...,1000,0.319,26,2,13,a18,170,615.0,862.0,2.0
307,6667060114_4,635.0,1250.0,890.0,1746.0,2494.0,1786.0,S2,3.0,3.0,...,1000,0.319,26,2,13,a18,170,615.0,856.0,2.0
308,6667060114_5,1244.0,1860.0,40.0,896.0,2494.0,1786.0,S3,3.0,3.0,...,1000,0.319,26,2,13,a18,170,616.0,856.0,2.0
309,6667060114_6,1244.0,1860.0,890.0,1746.0,2494.0,1786.0,S4,3.0,3.0,...,1000,0.319,26,2,13,a18,170,616.0,856.0,2.0


In [30]:
margin_cleaner(abin, df)
abin

checker: True
end of for
Awesome, no margin left! Lets update the df :)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

,DocName,x_left,x_right,y_bot,y_top,x_sheetsize,y_sheetsize,BSig,MarBot,MarLeft,...,amount,cut_time,cuts,layer,cut_layer,format,grammatur,width,height,new_layer
303,6667060114_0,1858.5,2467.5,41.5,894.5,2494.0,1786.0,X2,3.0,3.0,...,1000,0.319,26,2,13,a18,170,615.0,856.0,2.0
304,6667060114_1,1858.5,2467.5,891.5,1744.5,2494.0,1786.0,X3,3.0,3.0,...,1000,0.319,26,2,13,a18,170,615.0,856.0,2.0
305,6667060114_2,30.5,639.5,44.5,1744.5,2494.0,1786.0,R1,3.0,3.0,...,1000,0.319,26,2,13,a18,170,615.0,1703.0,2.0
306,6667060114_3,639.5,1248.5,35.5,894.5,2494.0,1786.0,Z1,3.0,3.0,...,1000,0.319,26,2,13,a18,170,615.0,862.0,2.0
307,6667060114_4,639.5,1248.5,891.5,1744.5,2494.0,1786.0,S2,3.0,3.0,...,1000,0.319,26,2,13,a18,170,615.0,856.0,2.0
308,6667060114_5,1248.5,1858.5,41.5,894.5,2494.0,1786.0,S3,3.0,3.0,...,1000,0.319,26,2,13,a18,170,616.0,856.0,2.0
309,6667060114_6,1248.5,1858.5,891.5,1744.5,2494.0,1786.0,S4,3.0,3.0,...,1000,0.319,26,2,13,a18,170,616.0,856.0,2.0


In [31]:
df[df['JobID']== 6667060114]

,DocName,x_left,x_right,y_bot,y_top,x_sheetsize,y_sheetsize,BSig,MarBot,MarLeft,...,amount,cut_time,cuts,layer,cut_layer,format,grammatur,width,height,new_layer
303,6667060114_0,1858.5,2467.5,41.5,hehe,2494.0,1786.0,X2,3.0,3.0,...,1000,0.319,26,2,13,a18,170,615.0,856.0,2.0
304,6667060114_1,1858.5,2467.5,891.5,hehe,2494.0,1786.0,X3,3.0,3.0,...,1000,0.319,26,2,13,a18,170,615.0,856.0,2.0
305,6667060114_2,30.5,639.5,44.5,hehe,2494.0,1786.0,R1,3.0,3.0,...,1000,0.319,26,2,13,a18,170,615.0,1703.0,2.0
306,6667060114_3,639.5,1248.5,35.5,hehe,2494.0,1786.0,Z1,3.0,3.0,...,1000,0.319,26,2,13,a18,170,615.0,862.0,2.0
307,6667060114_4,639.5,1248.5,891.5,hehe,2494.0,1786.0,S2,3.0,3.0,...,1000,0.319,26,2,13,a18,170,615.0,856.0,2.0
308,6667060114_5,1248.5,1858.5,41.5,hehe,2494.0,1786.0,S3,3.0,3.0,...,1000,0.319,26,2,13,a18,170,616.0,856.0,2.0
309,6667060114_6,1248.5,1858.5,891.5,hehe,2494.0,1786.0,S4,3.0,3.0,...,1000,0.319,26,2,13,a18,170,616.0,856.0,2.0


## margin_cut

In [ ]:
def margin_cut(abin, count_cut, list_actual_cut):

  max_width, max_height, min_height, min_width = 0, 0, M, M

  # iterriere über einzelnen bin, erhalte die max, min Werte
  for index, row in abin.iterrows():
    if row['x_right'] >= max_width:
      max_width = row['x_right']
    if row['x_left'] < min_width:
      min_width = row['x_left']
    if row['y_top'] >= max_height:
      max_height = row['y_top']
    if row['y_bot'] < min_height:
      min_height = row['y_bot']

  # erzeuge gerundete listen
  x_margin_cut = [max_width, min_width]
  y_margin_cut = [max_height, min_height]
  x_margin_cut = list((np.array(x_margin_cut)))
  y_margin_cut = list(np.around(np.array(y_margin_cut)))

  # print('\n--- max_width, max_height, min_height, min_width:', max_width, max_height, min_height, min_width)
  # print('\n--- x_margin_cut, y_margin_cut:', x_margin_cut, y_margin_cut)
  
  # schreibe margin_cuts in die geschnittenen abins
  # um beim rand schneiden zu erkennen, welche schnitte ausstehen
  for cut in x_margin_cut:
    # print('\n---cut in x_margin_cut:', cut)
    for index, row in abin.iterrows():
      
      if (((row['x_right']) == cut)):
        # print('\n--- BOOL MARGIN CUT vergleich x_right:', (((row['x_right']) == cut)))
        row['x_cut_right'] = cut
      elif (((row['x_left']) == cut)):
        # print('\n--- BOOL MARGIN CUT vergleich x_left:', (((row['x_left']) == cut)))
        row['x_cut_left'] = cut
  
    count_cut +=1
    list_actual_cut.append(x_margin_cut)

  for cut in y_margin_cut:
    for index, row in abin.iterrows():
      if (((row['y_bot']) == cut)):
          row['y_cut_bot'] = cut
      elif (((row['y_top']) == cut)):
          row['y_cut_top'] = cut
    
    count_cut +=1
    list_actual_cut.append(y_margin_cut)
    
  return x_margin_cut, y_margin_cut, count_cut, abin

# Guilloutine

## possible_cut

In [ ]:
# df_margin_cut rauswerfen
def possible_cut(abin, x_margin_cut, y_margin_cut):

  x_possible_cut = []
  y_possible_cut = []

  x_possible_cut = abin['x_left'].tolist() + abin['x_right'].tolist()
  y_possible_cut = abin['y_bot'].tolist() + abin['y_top'].tolist()
  x_possible_cut = pd.unique(x_possible_cut).tolist()
  y_possible_cut = pd.unique(y_possible_cut).tolist()

  # actual cut am anfang 'nan', weil initial, irrelevant
  x_actual_cut = abin['x_cut_left'].tolist() + abin['x_cut_right'].tolist()
  y_actual_cut = abin['y_cut_bot'].tolist() + abin['y_cut_top'].tolist()
  x_actual_cut = pd.unique(x_actual_cut).tolist()
  y_actual_cut = pd.unique(y_actual_cut).tolist()

  x_possible_cut = list(set(x_possible_cut) - set(x_actual_cut) - set(x_margin_cut))
  y_possible_cut = list(set(y_possible_cut) - set(y_actual_cut) - set(y_margin_cut))

#   print('-----------------x_possible_cut', x_possible_cut)
#   print('-----------------y_possible_cut', y_possible_cut)
#   print('-----------------x_actual_cut', x_actual_cut)
#   print('-----------------y_actual_cut', y_actual_cut)
#   print('-----------------x_margin_cut', x_margin_cut)
#   print('-----------------y_margin_cut', y_margin_cut)

  return x_possible_cut, y_possible_cut, x_actual_cut, y_actual_cut

## check_gcut

In [ ]:
def x_check_gcut(x_possible_cut, abin):

  # Initialisiere Liste zum Speichern der gcut
  x_list_gcut = []

  if x_possible_cut != []:

    array_possible_cut = np.array(x_possible_cut)

    # Loop über alle möglichen cut 
    for i in range(len(array_possible_cut)):
      bool_list = []
      float_to_check = array_possible_cut[i]
      # print('Element in 1_For-Loop:\n', float_to_check, '\n')
      
      # Loop über alle Flyer im jeweiligen abin, um zu prüfen, ob der mögliche cut zwischen coord der x_Achse liegt 
      for index, row in abin.iterrows():
        bool_list.append(((float_to_check > row['x_left']) & (float_to_check < row['x_right'])))

        # print('--------')
        # print('abin_row y_bot in 2_For-Loop:\n', row['y_bot'], '\n')
        # print('abin_row y_top in 2_For-Loop:\n', row['y_top'], '\n')
        # print('check_bot:', (float_to_check > row['y_bot']))
        # print('check_top:',(float_to_check < row['y_top']))
        # print('check_if_state:',((float_to_check > row['y_bot']) & (float_to_check < row['y_top'])))
        # print('bool_list:', bool_list)

      if not True in bool_list:
        x_list_gcut.append(float_to_check)

    x_list_gcut = pd.unique(x_list_gcut)

    # print('-----------------x_list_gcut', x_list_gcut)

  return x_list_gcut

In [ ]:
def y_check_gcut(y_possible_cut, abin):

  # Initialisiere Liste zum Speichern der gcut
  y_list_gcut = []

  if y_possible_cut != []:

    array_possible_cut = np.array(y_possible_cut)

    # Loop über alle möglichen cut 
    for i in range(len(array_possible_cut)):
      bool_list = []
      float_to_check = array_possible_cut[i]
      # print('Element in 1_For-Loop:\n', float_to_check, '\n')
      
      # Loop über alle Flyer im jeweiligen abin, um zu prüfen, ob der mögliche cut zwischen coord der y_Achse liegt 
      for index, row in abin.iterrows():
        bool_list.append(((float_to_check > row['y_bot']) & (float_to_check < row['y_top'])))

        # print('--------')
        # print('abin_row y_bot in 2_For-Loop:\n', row['y_bot'], '\n')
        # print('abin_row y_top in 2_For-Loop:\n', row['y_top'], '\n')
        # print('check_bot:', (float_to_check > row['y_bot']))
        # print('check_top:',(float_to_check < row['y_top']))
        # print('check_if_state:',((float_to_check > row['y_bot']) & (float_to_check < row['y_top'])))
        # print('bool_list:', bool_list)

      if not True in bool_list:
        y_list_gcut.append(float_to_check)

    y_list_gcut = pd.unique(y_list_gcut)

    # print('-----------------y_list_gcut', y_list_gcut)

  return y_list_gcut

## prio

In [ ]:
def prio_gcut(abin, x_list_gcut, y_list_gcut):
  x_df_prio = pd.DataFrame(
            {'x_gcut': x_list_gcut,
            'y_gcut': 0,
            })
  
  y_df_prio = pd.DataFrame(
            {'x_gcut': 0,
            'y_gcut': y_list_gcut,
            })
  
  df_prio = x_df_prio.append(y_df_prio)
  df_prio[['touch_flyer','prio']] = 0

  # iterieren über den erzeugten df mit den gcuts
  for index, row in df_prio.iterrows():
    i = index
    # print('1_Koordinaten lt row[] aus df_prio:', row['x_gcut'], row['y_gcut'], row['touch_flyer'])
  
    # für jeden neuen gcut reset counter auf 0
    touch_flyer = 0

    # speichern der Koordinaten in einem Tupel zum Vergleich
    if ((row['x_gcut'] > 0) & (row['y_gcut'] > 0)):
      to_check = 'error'
    else:
      to_check = (row['x_gcut'], row['y_gcut']) 

    # iterieren über den abin
    for index, row in abin.iterrows():
    
      # prüfen ob es sich um x_gcut oder y_gcut handelt
      if to_check[0] > 0:
        # print('start vertikale if')
      
        # prüfen ob to_check (vertikaler schnitt) rechtecke berührt
        if ((to_check[0] == row['x_left']) | (to_check[0] == row['x_right'])):
          touch_flyer += 1
          # print('verikaler schnitt berührt:', touch_flyer)

      # prüfen ob es sich um x_gcut oder y_gcut handelt  
      elif to_check[1] > 0:
        # print('start horizontaler if')

        # prüfen ob to_check (horizontaler schnitt) rechtecke berührt
        if ((to_check[1] == row['y_bot']) | (to_check[1] == row['y_top'])):
          touch_flyer += 1
          # print('horizontaler schnitt berührt:', touch_flyer)

      # Fehlermeldung
      else:
        print('error compare to_check')

    # print('1_FLOOP', touch_flyer)
    df_prio.at[i,'touch_flyer'] = touch_flyer

  df_prio['prio'] = df_prio['touch_flyer'].rank(ascending=False)

  # print('\n\n-------------------df_prio\n\n', df_prio)

  return df_prio

## sub_bin_maker

In [ ]:
def sub_bin_maker(abin, df_prio):
  df_gcut = df_prio.iloc[df_prio['prio'].idxmin()]
  gcut = (df_gcut['x_gcut'], df_gcut['y_gcut'])

  # print('-----------------gcut', gcut)

  col = abin.columns

  i = 0

  # prüfen ob links/rechts von x_gcut
  if gcut[0] > 0:
    df_sbin_left = pd.DataFrame(columns=col)
    df_sbin_right = pd.DataFrame(columns=col)

    # for Schleife innerhalb if, da Prüfung gcut nur einmalig erforderlich
    for index, row in abin.iterrows():

      if (((row['x_left']) < gcut[0]) | ((row['x_right']) < gcut[0])):
        df_sbin_left.loc[i,:] = row[col]
        if (((row['x_right']) == gcut[0])):
          df_sbin_left.loc[i,'x_cut_right'] = gcut[0]
        elif (((row['x_left']) == gcut[0])):
          df_sbin_left.loc[i,'x_cut_left'] = gcut[0]
    
      elif (((row['x_left']) > gcut[0]) | ((row['x_right']) > gcut[0])):
        df_sbin_right.loc[i,:] = row[col]
        if (((row['x_right']) == gcut[0])):
          df_sbin_right.loc[i,'x_cut_right'] = gcut[0]
        elif (((row['x_left']) == gcut[0])):
          df_sbin_right.loc[i,'x_cut_left'] = gcut[0]
    
      i += 1
    # print('\n---Das ist Shape df_sbin_left:', df_sbin_left.shape)
    # print('\n---Das ist Shape df_sbin_right:', df_sbin_right.shape)

  # prüfen ob oben/unten von y_gcut
  elif gcut[1] > 0:
    df_sbin_bot = pd.DataFrame(columns=col)
    df_sbin_top = pd.DataFrame(columns=col)

    # for Schleife innerhalb if, da Prüfung gcut nur einmalig erforderlich
    for index, row in abin.iterrows():

      if (((row['y_bot']) < gcut[1]) | ((row['y_top']) < gcut[1])):
        df_sbin_bot.loc[i,:] = row[col]
        if (((row['y_bot']) == gcut[1])):
          df_sbin_bot.loc[i,'y_cut_bot'] = gcut[1]
        elif (((row['y_top']) == gcut[1])):
          df_sbin_bot.loc[i,'y_cut_top'] = gcut[1]
          
      elif (((row['y_bot']) > gcut[1]) | ((row['y_top']) > gcut[1])):
        df_sbin_top.loc[i,:] = row[col]
        if (((row['y_bot']) == gcut[1])):
          df_sbin_top.loc[i,'y_cut_bot'] = gcut[1]
        elif (((row['y_top']) == gcut[1])):
          df_sbin_top.loc[i,'y_cut_top'] = gcut[1]
    
      i += 1
    # print('\n---Das ist Shape df_sbin_bot:', df_sbin_bot.shape)
    # print('\n---Das ist Shape df_sbin_top:', df_sbin_top.shape)

  # Fehlermeldung
  else:
    print('error in sub_bin_maker')

  if gcut[0] > 0:
    return df_sbin_left, df_sbin_right, gcut
  elif gcut[1] > 0:
    return df_sbin_bot, df_sbin_top, gcut
  else:
    print('error in sub_bin_maker')

# Edgy

## to_edge_cut

In [ ]:
def to_edge_cut(abin, x_margin_cut, y_margin_cut, x_actual_cut, y_actual_cut):
  
  x_needed_cut = []
  y_needed_cut = []
  
  x_needed_cut = abin['x_left'].tolist() + abin['x_right'].tolist() 
  y_needed_cut = abin['y_bot'].tolist() + abin['y_top'].tolist()

  x_actual_cut = abin['x_cut_right'].tolist() + abin['x_cut_left'].tolist() 
  y_actual_cut = abin['y_cut_bot'].tolist()+ abin['y_cut_top'].tolist()

  x_edge_cut = list(set(x_needed_cut) - set(x_actual_cut) - set(x_margin_cut))
  y_edge_cut = list(set(y_needed_cut) - set(y_actual_cut) - set(y_margin_cut))
  
#   print('\n\n-------no_edge--x_edge_cut & y_edge_cut:', x_edge_cut, y_edge_cut)
#   print('\n\n-------no_edge--actual_cut:', x_actual_cut, y_actual_cut)
#   print('\n\n-------no_edge--abin:\n\n', abin)

  return x_edge_cut, y_edge_cut

## no_edge

In [ ]:
def no_edge(abin, list_actual_cut, count_cut, x_edge_cut, y_edge_cut):

  for cut in x_edge_cut:
    for index, row in abin.iterrows():
      if (((row['x_right']) == cut)):
        row['x_cut_right'] = cut
      elif (((row['x_left']) == cut)):
        row['x_cut_left'] = cut
      else:
        # print('error in no_edger')
        continue
    
    count_cut +=1
    list_actual_cut.append(x_edge_cut)

  for cut in y_edge_cut:
    for index, row in abin.iterrows():
      if (((row['y_bot']) == cut)):
          row['y_cut_bot'] = cut
      elif (((row['y_top']) == cut)):
          row['y_cut_top'] = cut
      else:
        # print('error in no_edger')
        continue
    
    count_cut +=1
    list_actual_cut.append(y_edge_cut)

  return abin, list_actual_cut, count_cut

# Masterfunction


In [1]:
def master_function(df):

  df['count_cut'] = 'nan'

  list_JobID = np.unique(df['JobID']).tolist()

  for id in list_JobID[:5]:
    print('~~~ -------------------------------')
    print('\n\n~~~ ID:', id)
    abin = get_abin(df, id)

    # Deklarieren
    abin['x_cut_left'] = 'nan'
    abin['x_cut_right'] = 'nan'
    abin['y_cut_bot'] = 'nan'
    abin['y_cut_top'] = 'nan'
    count_cut = 0
    max_bin_len = 0 
    count_break = 0
    list_actual_cut = []
    x_possible_cut = []
    y_possible_cut = []
    x_edge_cut = []
    y_edge_cut = []

    x_margin_cut, y_margin_cut, count_cut, abin = margin_cut(abin, count_cut, list_actual_cut)
    # print('~~~ abin-nach-margin_cut:\n\n', abin)
    print('~~~ list_actual_cut\n\n', list_actual_cut)

    if abin.shape[0] != 1:
      
    ###   x_overlap = x_calc_overlap(abin)
    ###   y_overlap = y_calc_overlap(abin)

    ###   if x_overlap != []:
    ###     x_no_margin, abin, df = calc_no_margin(list_overlap=x_overlap, abin=abin, df=df)
      
    ###   if y_overlap != []:
    ###     y_no_margin, abin, df = calc_no_margin(list_overlap=y_overlap, abin=abin, df=df)

      # ähnliche funktion in calc_no_margin, nur eine nutzen!
      # im df aktuellen abin, durch neuen abin mit harmonisierten Werten ersetzen
      # df.loc[['x_left', 'x_right',	'y_bot',	'y_top']] = abin[['x_left', 'x_right',	'y_bot',	'y_top']]
      # print('\n~~~ df nach ersetzen:\n\n', df)


      # inkl. SUB-BIN
      list_df_bin = [abin]
      

      # solange irgendein df in der liste >1 element ist muss gecuttet werden
      while max_bin_len != 1:

        # reset max_bin_len, muss bei jedem Durchgang neu getestet werden
        max_bin_len = 0 
        count_break += 1

        # Abbruchbedingung max_bin_len und pop des wirklich längsten df_bin
        for i , abin in enumerate(list_df_bin):
          df_bin = list_df_bin[i]
          bin_len = df_bin.shape[0]
          # Maximum setzen und Listenindex in pop_ix
          if bin_len > max_bin_len:
            max_bin_len = bin_len
            pop_ix = i
          
          # print('\n~~~ all found max_bin_len in list_df_bin:', max_bin_len)
          # print('\n~~~ all found pop_ix in list_df_bin:', pop_ix)

        longest_bin = list_df_bin.pop(pop_ix)
        # print('\n~~~ Shape longest_bin:', longest_bin.shape)

        # Funktionen in der Loop
        x_possible_cut, y_possible_cut, x_actual_cut, y_actual_cut = possible_cut(abin=longest_bin, x_margin_cut=x_margin_cut, y_margin_cut=y_margin_cut)

        if ((x_possible_cut == []) & (y_possible_cut == [])):
          # print('ERROR ~~~ no possible cuts')
          break

        x_gcut = x_check_gcut(x_possible_cut=x_possible_cut, abin=longest_bin)
        y_gcut = y_check_gcut(y_possible_cut=y_possible_cut, abin=longest_bin)

        # variablen namen aufeinander besser abstimmen
        df_prio = prio_gcut(abin=longest_bin, x_list_gcut=x_gcut, y_list_gcut=y_gcut)

        # speichern der sbin
        sbin_1, sbin_2, actual_cut = sub_bin_maker(abin=longest_bin, df_prio=df_prio)

        # print('\n~~~ Shape sbin_1:', sbin_1.shape)
        # print('\n~~~ Shape sbin_2:', sbin_2.shape)
        # print('\n~~~ Len list_df_bin - PRE sbin append:', len(list_df_bin))

        if sbin_1.shape[0] > 0:
          list_df_bin.append(sbin_1)

        if sbin_2.shape[0] > 0:
          list_df_bin.append(sbin_2)

        list_actual_cut.append(actual_cut)

        # print('\n~~~ Len list_df_bin - POST sbin append:', len(list_df_bin))

        sbin_1 = ''
        sbin_2 = ''
        count_cut +=1
        # print('\n\n~~~ count_cut:', count_cut)
        # print('~~~ LAST STATEMENT IN WHILE LOOP -------------------\n\n')

        # if count_break == 3: break
      # END of WHILE

      # check if all element in list are length 1
      # sum_bin_len = 0
      # bin_len = 0

      for abin in (list_df_bin):
        if abin.shape[0] != 1:
          print('ERROR ERROR ERROR bei Randschneiden, ein bin ist >1')

      # Rand schneiden - Iteration über list_df_bin, wenn alle Elemente = 1
      for n , abin in enumerate(list_df_bin):
        # print('\n\n~~~ RANDSCHNEIDEN:\n', abin)
        x_edge_cut, y_edge_cut = to_edge_cut(abin=abin, x_margin_cut=x_margin_cut, y_margin_cut=y_margin_cut, x_actual_cut=x_actual_cut, y_actual_cut=y_actual_cut)

        if ((x_edge_cut != []) | (y_edge_cut != [])):
          # Aufruf der Cutting-Functions zum Ränder schneiden
          # x_possible_cut, y_possible_cut, x_actual_cut, y_actual_cut = possible_cut(abin=abin, x_margin_cut=x_margin_cut, y_margin_cut=y_margin_cut, x_edge_cut=x_edge_cut, y_edge_cut=y_edge_cut)

          # x_gcut = x_check_gcut(x_possible_cut=x_possible_cut, abin=abin)
          # y_gcut = y_check_gcut(y_possible_cut=y_possible_cut, abin=abin)

          # cutten der Ränder
          abin, list_actual_cut, count_cut = no_edge(abin=abin, list_actual_cut=list_actual_cut, count_cut=count_cut, x_edge_cut=x_edge_cut, y_edge_cut=y_edge_cut)
          list_df_bin[n] = abin

    # schreibe count_cut in neue spalte des ausgangs df
    # noch in erster for schleife von oben
    df.loc[df['JobID'] == id, 'count_cut'] = count_cut
    # print('\n\n~~~ Hier werden counts gesetzt:\n\n', df.loc[df['JobID'] == id])

    # weitere print statements, um die Ergebnisse der Funktionen zu prüfen
    # print('-------------------------')
    # print('x_margin_cut:\n', x_margin_cut, '\n')
    # print('y_margin_cut:\n', y_margin_cut, '\n')
    # print('x_no_margin:\n', x_no_margin, '\n')
    # print('y_no_margin:\n', y_no_margin, '\n')
    # print('~~~')
    # print('x_possible_cut:\n', x_possible_cut, '\n')
    # print('y_possible_cut:\n', y_possible_cut, '\n')
    # print('x_gcut:\n', x_gcut, '\n')
    # print('y_gcut:\n', y_gcut, '\n')
    # print('abin:\n', abin, '\n')

  return df

In [ ]:
df = master_function(df)

# Export

In [ ]:
# df.to_csv('/content/drive/MyDrive/PJS/01_Input/Daten_v1_clean/01_Preprocessed/prep_flyerdata_135_gl_b17.csv')

# Runtime

In [ ]:
# %timeit for x in range(100): master_function(df)

In [ ]:
# timeit(master_function(df))